---
title: "Example Notebooks of SPICE Toolkit" \
author: "Naoya Ozaki" \
date: "26 November 2022" \
output: "spice_example"

---

## Plot Moon's Orbit
Import SPICE library (download generic kernels if needed) and obtain the position and velocity of the moon.
The download of the generic kernels may take long time because of downloading the large file (~120MB).

In [1]:
# Import module
using Downloads

function get_path_of_genker(kernel_name::String)::String
    """Get the path of specific generic kernels.

    Args:
        kernel_name (String): Name of the generic kernels

    Returns:
        String: Full path of the generic kernels

    Note:
        The downloaded generic kernels are supposed to 
        be stored in <current_directory> * "/../data/lib"
    """
    # Get the spice directory
    current_directory = dirname(abspath(@__FILE__))
    spice_directory = current_directory * "/../data/lib"

    # Split the input string
    kernel_dir = split(kernel_name, "/")

    # Find naif0012.tls in the directory
    is_downloaded = false
    genker_path = ""
    for (root, dirs, files) in walkdir(spice_directory)
        is_downloaded = any(occursin.(kernel_dir[end], files))
        if is_downloaded
            genker_path = root * "/" * kernel_dir[end]
            break
        end
    end
    if !is_downloaded
        mkpath(dirname(spice_directory * "/" * kernel_name))
        genker_path = Downloads.download("https://naif.jpl.nasa.gov/pub/naif/generic_kernels/" * kernel_name, spice_directory * "/" * kernel_name)
    end

    return genker_path
end

get_path_of_genker (generic function with 1 method)

In [2]:
import SPICE
# Load generic kernels
SPICE.furnsh(get_path_of_genker("lsk/naif0012.tls")) # Leap seconds kernel
SPICE.furnsh(get_path_of_genker("pck/gm_de431.tpc")) # Gravity Constant
SPICE.furnsh(get_path_of_genker("spk/planets/de440.bsp")) # Planetary ephemeris kernel

In [3]:
# Parameter Setting
et0 = SPICE.str2et("2021/04/06 13:52:32 UTC") # Initial Epoch
etf = SPICE.str2et("2022/04/06 13:52:32 UTC") # Final Epoch
num_grid = 10000 # Number of Grid

# Pre-calculation
et_all = LinRange(et0,etf,num_grid)
x_moon = zeros(num_grid, 6)

# Iteration
for i = 1:num_grid
    x_moon[i,:], _ = SPICE.spkez(301, et_all[i], "ECLIPJ2000", "NONE", 399)
end

10000×6 Matrix{Float64}:
      2.64279e5       -2.80462e5  -30110.1   0.768808  0.660662  -0.0473644
      2.66694e5       -2.78368e5  -30258.5   0.76299   0.666869  -0.0466959
      2.69091e5       -2.76255e5  -30404.7   0.757127  0.673023  -0.0460249
      2.7147e5   -274123.0        -30548.8   0.75122   0.679122  -0.0453515
 273830.0             -2.71972e5  -30690.7   0.745268  0.685168  -0.0446758
      2.76171e5       -2.69801e5  -30830.6   0.739273  0.691158  -0.0439978
      2.78493e5       -2.67612e5  -30968.3   0.733235  0.697094  -0.0433176
      2.80796e5  -265404.0        -31103.8   0.727154  0.702975  -0.0426351
      2.8308e5        -2.63178e5  -31237.2   0.721031  0.708801  -0.0419505
      2.85344e5       -2.60933e5  -31368.4   0.714866  0.714571  -0.0412638
      ⋮                                                           ⋮
      1.13895e5        3.86411e5   13089.7  -0.924245  0.29097    0.0829888
      1.10977e5        3.87316e5   13351.0  -0.926366  0.283427   0.082

In [4]:
using Plots
plot(x_moon[:, 1], x_moon[:, 2], x_moon[:, 3], xlim=[-5e5, 5e5], ylim=[-5e5, 5e5], zlim=[-5e5, 5e5], aspect_ratio=:equal)